<a href="https://colab.research.google.com/github/susan291-gifs/SussieAssignment/blob/main/Ensemble_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

###Problem 1

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
X = data[['GrLivArea', 'YearBuilt']].values
y = data['SalePrice'].values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

models = [LinearRegression(), LinearRegression(), LinearRegression()]

subset_size = len(X_train) // len(models)
for i, model in enumerate(models):
    start_idx = i * subset_size
    end_idx = start_idx + subset_size
    model.fit(X_train[start_idx:end_idx], y_train[start_idx:end_idx])

predictions = np.zeros((len(X_val), len(models)))
for i, model in enumerate(models):
    predictions[:, i] = model.predict(X_val)

blending_model = LinearRegression()
blending_model.fit(predictions, y_val)

final_prediction = blending_model.predict(predictions)

mse = mean_squared_error(y_val, final_prediction)
print("Mean Squared Error (MSE) on validation data:", mse)


Mean Squared Error (MSE) on validation data: 2418849078.3184934


###Problem 2

In [ ]:
class BaggingRegressor:
    def __init__(self, n_estimators):
        self.n_estimators = n_estimators
        self.models = []

    def fit(self, X, y):
        for _ in range(self.n_estimators):
            model = DecisionTreeRegressor()
            indices = np.random.choice(len(X), len(X), replace=True)
            X_subset, y_subset = X[indices], y[indices]
            model.fit(X_subset, y_subset)
            self.models.append(model)

    def predict(self, X):
        predictions = np.zeros((X.shape[0], len(self.models)))
        for i, model in enumerate(self.models):
            predictions[:, i] = model.predict(X)
        return np.mean(predictions, axis=1)


data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
X = data[['GrLivArea', 'YearBuilt']].values
y = data['SalePrice'].values


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


bagging_regressor = BaggingRegressor(n_estimators=10)
bagging_regressor.fit(X_train, y_train)


y_pred_bagging = bagging_regressor.predict(X_val)


mse_bagging = mean_squared_error(y_val, y_pred_bagging)
print("Mean Squared Error (MSE) using Bagging:", mse_bagging)

Mean Squared Error (MSE) using Bagging: 1683433222.9288468


###Problem 3

In [ ]:
class StackingRegressor:
    def __init__(self, base_models, meta_model):
        self.base_models = base_models
        self.meta_model = meta_model

    def fit(self, X, y):
        self.meta_features = np.zeros((len(X), len(self.base_models)))
        for i, model in enumerate(self.base_models):
            model.fit(X, y)
            self.meta_features[:, i] = model.predict(X)
        self.meta_model.fit(self.meta_features, y)

    def predict(self, X):
        meta_features = np.zeros((len(X), len(self.base_models)))
        for i, model in enumerate(self.base_models):
            meta_features[:, i] = model.predict(X)
        return self.meta_model.predict(meta_features)

data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
X = data[['GrLivArea', 'YearBuilt']].values
y = data['SalePrice'].values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

base_models = [DecisionTreeRegressor(), SVR(), LinearRegression()]

meta_model = LinearRegression()

stacking_regressor = StackingRegressor(base_models, meta_model)
stacking_regressor.fit(X_train, y_train)

y_pred_stacking = stacking_regressor.predict(X_val)


mse_stacking = mean_squared_error(y_val, y_pred_stacking)
print("Mean Squared Error (MSE) using Stacking:", mse_stacking)

Mean Squared Error (MSE) using Stacking: 2077433603.7752101
